# Loop 5 Analysis: Understanding the Gap to Target

## Key Questions:
1. What is the actual gap between best public solution (70.676) and target (68.923)?
2. What techniques have NOT been tried yet?
3. What are the correct SA parameters from jiweiliu?

In [1]:
import json
import numpy as np
import pandas as pd

# Load session state
with open('/home/code/session_state.json') as f:
    state = json.load(f)

print('=== EXPERIMENT SUMMARY ===')
for exp in state['experiments']:
    print(f"{exp['id']}: {exp['name']} - Score: {exp['score']:.6f}")

print('\n=== SUBMISSIONS ===')
for sub in state['submissions']:
    print(f"{sub['model_name']}: CV={sub['cv_score']:.6f}, LB={sub['lb_score']}")

print('\n=== KEY FINDINGS ===')
for finding in state['data_findings'][-5:]:
    print(f"- {finding['finding'][:200]}...")

=== EXPERIMENT SUMMARY ===
exp_000: 001_baseline_preoptimized - Score: 70.676102
exp_001: 002_ensemble_all_sources - Score: 70.676102
exp_002: 003_lattice_backward_prop - Score: 70.676102
exp_003: 004_numba_backward_prop - Score: 70.676102
exp_004: 005_full_sa_lattice - Score: 70.676102

=== SUBMISSIONS ===
001_baseline_preoptimized: CV=70.676102, LB=70.676102398091
002_ensemble_all_sources: CV=70.676102, LB=

=== KEY FINDINGS ===
- Score breakdown: Small N (1-50) contributes 19.04, Medium N (51-100) contributes 17.64, Large N (101-200) contributes 33.99. Large N has most room for improvement. Efficiency is 80.6% for small N, 83....
- The egortrushin kernel uses lattice-based construction: (1) Create 2-tree unit cells (one up, one down), (2) Tile with fractional translations (dx, dy), (3) Apply SA to optimize translation parameters...
- Rebuild from corners technique found 1 improvement: N=54 improved from 4.41716128 to 4.41328175 (0.000634 total score improvement). This technique extr

In [2]:
# Calculate the gap
best_score = 70.676102
target = 68.922808
gap = best_score - target
gap_pct = gap / target * 100

print(f'Best Score: {best_score:.6f}')
print(f'Target: {target:.6f}')
print(f'Gap: {gap:.6f} ({gap_pct:.2f}%)')
print()
print('This is a 2.5% improvement needed - significant but achievable.')

Best Score: 70.676102
Target: 68.922808
Gap: 1.753294 (2.54%)

This is a 2.5% improvement needed - significant but achievable.


In [3]:
# Analyze what the jiweiliu kernel ACTUALLY does vs what we tried
print('=== JIWEILIU SA PARAMETERS (CORRECT) ===')
jiweiliu_params = {
    'Tmax': 0.001,
    'Tmin': 0.000001,
    'nsteps': 10,
    'nsteps_per_T': 10000,
    'position_delta': 0.002,
    'angle_delta': 1.0,
    'angle_delta2': 1.0,
    'delta_t': 0.002,
}

print('\n=== OUR EXPERIMENT PARAMETERS (WRONG) ===')
our_params = {
    'Tmax': 0.1,
    'Tmin': 1e-6,
    'nsteps': 5000,
    'nsteps_per_T': 100,
    'position_delta': 0.1,
    'angle_delta': 10.0,
    'angle_delta2': 5.0,
    'delta_t': 0.05,
}

print('\n=== PARAMETER COMPARISON ===')
for key in jiweiliu_params:
    ratio = our_params[key] / jiweiliu_params[key]
    print(f'{key}: jiweiliu={jiweiliu_params[key]}, ours={our_params[key]}, ratio={ratio:.1f}x')

=== JIWEILIU SA PARAMETERS (CORRECT) ===

=== OUR EXPERIMENT PARAMETERS (WRONG) ===

=== PARAMETER COMPARISON ===
Tmax: jiweiliu=0.001, ours=0.1, ratio=100.0x
Tmin: jiweiliu=1e-06, ours=1e-06, ratio=1.0x
nsteps: jiweiliu=10, ours=5000, ratio=500.0x
nsteps_per_T: jiweiliu=10000, ours=100, ratio=0.0x
position_delta: jiweiliu=0.002, ours=0.1, ratio=50.0x
angle_delta: jiweiliu=1.0, ours=10.0, ratio=10.0x
angle_delta2: jiweiliu=1.0, ours=5.0, ratio=5.0x
delta_t: jiweiliu=0.002, ours=0.05, ratio=25.0x


In [4]:
# What approaches have been tried?
print('=== APPROACHES TRIED ===')
approaches = [
    ('001_baseline', 'Pre-optimized saspav solution', 'No improvement possible'),
    ('002_ensemble', 'Ensemble from multiple sources', 'saspav already best for all N'),
    ('003_lattice', 'Lattice construction + backward prop', 'Worse than baseline'),
    ('004_numba_sa', 'Numba backward propagation', '0 improvements'),
    ('005_full_sa', 'Full SA with WRONG parameters', 'Worse than baseline'),
]

for name, desc, result in approaches:
    print(f'{name}: {desc} -> {result}')

print('\n=== APPROACHES NOT YET TRIED ===')
not_tried = [
    'SA with CORRECT jiweiliu parameters (Tmax=0.001, position_delta=0.002)',
    'C++ tree_packer_v18 optimizer (from jonathanchan kernel)',
    'Parallel grid exploration (all viable grid sizes)',
    'Fractional translation optimization',
    'Extended grid with append_x/append_y',
]
for approach in not_tried:
    print(f'- {approach}')

=== APPROACHES TRIED ===
001_baseline: Pre-optimized saspav solution -> No improvement possible
002_ensemble: Ensemble from multiple sources -> saspav already best for all N
003_lattice: Lattice construction + backward prop -> Worse than baseline
004_numba_sa: Numba backward propagation -> 0 improvements
005_full_sa: Full SA with WRONG parameters -> Worse than baseline

=== APPROACHES NOT YET TRIED ===
- SA with CORRECT jiweiliu parameters (Tmax=0.001, position_delta=0.002)
- C++ tree_packer_v18 optimizer (from jonathanchan kernel)
- Parallel grid exploration (all viable grid sizes)
- Fractional translation optimization
- Extended grid with append_x/append_y


In [5]:
# Check what C++ optimizers are available
import os

print('=== AVAILABLE C++ OPTIMIZERS ===')
optimizers = [
    '/home/code/bbox3_advanced',
    '/home/code/tree_packer',
    '/home/code/tree_packer_v18',
]

for opt in optimizers:
    if os.path.exists(opt):
        size = os.path.getsize(opt)
        print(f'{opt}: {size/1024:.1f} KB')
    else:
        print(f'{opt}: NOT FOUND')

=== AVAILABLE C++ OPTIMIZERS ===
/home/code/bbox3_advanced: 120.8 KB
/home/code/tree_packer: 221.7 KB
/home/code/tree_packer_v18: 193.5 KB


In [6]:
# Key insight: The jiweiliu kernel claims ~0.15 improvement
# But we need 1.75 points improvement
# This means we need to:
# 1. Run SA with correct parameters
# 2. Explore ALL grid configurations (not just 4)
# 3. Use C++ optimizer for speed
# 4. Combine multiple techniques

print('=== IMPROVEMENT NEEDED ===')
print(f'Gap to target: {gap:.6f}')
print(f'jiweiliu claims: ~0.15 improvement')
print(f'Remaining gap after jiweiliu: {gap - 0.15:.6f}')
print()
print('This suggests we need MULTIPLE techniques combined:')

=== IMPROVEMENT NEEDED ===
Gap to target: 1.753294
jiweiliu claims: ~0.15 improvement
Remaining gap after jiweiliu: 1.603294

This suggests we need MULTIPLE techniques combined:
